In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
# !pip install coloredlogs onnx onnxruntime

In [3]:
from flair.models import TextClassifier
from flair.embeddings import TransformerWordEmbeddings, TransformerDocumentEmbeddings
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus, Sentence

In [4]:
model = TextClassifier.load("models/rubert_1731798298/model_epoch_3.pt")

In [5]:
assert isinstance(model.embeddings, (TransformerWordEmbeddings, TransformerDocumentEmbeddings))

In [6]:
column_name_map = {0: "text", 1: "label"}
corpus: Corpus = CSVClassificationCorpus("flair_data",
                                            column_name_map,
                                            skip_header=False,
                                            delimiter='\t',    # tab-separated files
                                            label_type='label')

2024-11-16 23:40:57,371 Reading data from flair_data
2024-11-16 23:40:57,373 Train: flair_data/train.csv
2024-11-16 23:40:57,375 Dev: flair_data/dev.csv
2024-11-16 23:40:57,376 Test: flair_data/test.csv


In [7]:
test = list(corpus.test)[:100]

In [8]:
model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label') # 2 30 min

  5%|▍         | 3/63 [00:07<02:30,  2.51s/it]

KeyboardInterrupt



In [9]:
model.embeddings = model.embeddings.export_onnx("embeddings-rubert-base.onnx", test[:10], providers=["CPUExecutionProvider"], session_options={})

In [10]:
model.save("model_onnx.pt")

In [11]:
model = TextClassifier.load("model_onnx.pt")

In [12]:
result = model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label') # 3 30 min

 29%|██▊       | 18/63 [00:05<00:14,  3.21it/s]


KeyboardInterrupt: 

In [10]:
model.embeddings.quantize_model(
            "embeddings-rubert-base-optimized-8bit.onnx",  extra_options={"DisableShapeInference": True}, use_external_data_format=False
        )

In [15]:
model.embeddings.remove_session()
model.embeddings.onnx_model = "embeddings-rubert-base-optimized-8bit.onnx"
model.embeddings.providers = ["CPUExecutionProvider", "GPUExecutionProvider"] # updated providers config
model.embeddings.create_session()

*************** EP Error ***************
EP Error Unknown Provider Type: GPUExecutionProvider when using ['CPUExecutionProvider', 'GPUExecutionProvider']
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************


In [16]:
result = model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label')

 35%|███▍      | 22/63 [00:06<00:12,  3.18it/s]

KeyboardInterrupt



In [16]:
print(result.detailed_results)


Results:
- F-score (micro) 0.998
- F-score (macro) 0.9988
- Accuracy 0.998

By class:
                precision    recall  f1-score   support

NON_FOOD_GOODS     1.0000    0.9896    0.9948        96
    FOOD_GOODS     1.0000    1.0000    1.0000        90
       SERVICE     0.9888    1.0000    0.9944        88
  BANK_SERVICE     1.0000    1.0000    1.0000        49
           TAX     1.0000    1.0000    1.0000        48
          LOAN     1.0000    1.0000    1.0000        41
       LEASING     1.0000    1.0000    1.0000        38
   REALE_STATE     1.0000    1.0000    1.0000        27
NOT_CLASSIFIED     1.0000    1.0000    1.0000        23

      accuracy                         0.9980       500
     macro avg     0.9988    0.9988    0.9988       500
  weighted avg     0.9980    0.9980    0.9980       500



In [17]:
model.save("model.pt")

In [18]:
model = TextClassifier.load("model.pt")

In [19]:
model.evaluate(corpus.test, verbose=True, mini_batch_size=8, gold_label_type='label')

 25%|██▌       | 16/63 [00:04<00:14,  3.30it/s]

KeyboardInterrupt



In [ ]:
model.embeddings.onnx_model